In [43]:
import requests
import pandas as pd
from pathlib import Path
from pandas import json_normalize
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import numpy as np
import re
import json
from rapidfuzz import process, fuzz
import unicodedata

In [44]:
#!pip install -q ollama

# Import and create a client that points explicitly to the local daemon
from ollama import Client

# Explicit host helps avoid accidental DNS problems
client = Client(host="http://127.0.0.1:11434")   # <- change if your server runs elsewhere

# Pick a model that you have already pulled locally
model_name = "gpt-oss:120b-cloud"

In [45]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"

In [46]:
periodos = ['2022-2026'] # <- Elegir períodos
df_par = pd.read_csv(DATA_DIR / 'parlamentarios.csv')

In [48]:
HEADERS = {"User-Agent": "Mozilla/5.0"}

def hybrid_match_nombre(nombre, lista_nombres):
    """
    Busca el nombre más similar combinando token_set_ratio y partial_token_set_ratio.
    """
    mejores = process.extract(
        nombre,
        lista_nombres,
        scorer=fuzz.token_set_ratio,
        limit=5
    )

    # Recalcula con partial_token_set_ratio para cada candidato
    mejor_nombre = None
    mejor_score = 0

    for candidato, score1, _ in mejores:
        score2 = fuzz.partial_token_set_ratio(nombre, candidato)
        score_final = max(score1, score2)  # o podrías usar promedio: (score1 + score2) / 2
        if score_final > mejor_score:
            mejor_score = score_final
            mejor_nombre = candidato

    return mejor_nombre, mejor_score


def normalize_name(name):
    if pd.isna(name):
        return ""
    name = name.strip().lower()
    name = " ".join(name.split())  # quita dobles espacios
    name = ''.join(
        c for c in unicodedata.normalize('NFD', name)
        if unicodedata.category(c) != 'Mn'  # elimina acentos
    )
    return name

def norm(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip(" \t\n\r")

def fetch_html(url: str, sleep_=0.25):
    r = requests.get(url, headers=HEADERS, timeout=30)
    sleep(sleep_)
    return r.status_code, r.text

def get_section_paragraphs(soup: BeautifulSoup, title_patterns):
    """
    Busca div.box_contenidos con <h4> cuyo texto calce con alguno de title_patterns (regex o strings),
    y devuelve una lista de párrafos (texto limpio) dentro de ese box.
    """
    if not isinstance(title_patterns, (list, tuple)):
        title_patterns = [title_patterns]

    paras = []
    for box in soup.select("div.box_contenidos"):
        h4 = box.find("h4")
        if not h4:
            continue
        title = norm(h4.get_text(" ", strip=True))
        if any(re.search(p, title, flags=re.I) for p in title_patterns):
            # en tu HTML: h4 + div con <p>… Tomamos todos los <p> dentro del box.
            for p in box.find_all("p"):
                t = norm(p.get_text(" ", strip=True))
                if t:
                    paras.append(t)
    return paras

def extract_paragraphs_from_url(url: str, sleep_=0.25):
    out = {
        "status": None,
        "familia_juventud_parrafos": [],
        "estudios_vida_laboral_parrafos": [],
    }
    status, html = fetch_html(url)
    out["status"] = status
    if status != 200:
        return out
    soup = BeautifulSoup(html, "html.parser")
    
    resultados = []
    for td in soup.find_all("td", class_="trayectoria_align"):
        texto_cargo = td.get_text(" ", strip=True)

        # --- 1. Extraer período robusto (acepta texto o meses en el año final) ---
        match_periodo = re.search(r"(\d{4})\s*[-–]\s*[A-Za-z]*\s*(\d{4})", texto_cargo)
        if not match_periodo:
            continue

        periodo = f"{match_periodo.group(1)}-{match_periodo.group(2)}"
        if periodo not in periodos:
            continue

        # --- 2. Extraer distrito ---
        distrito_num = None

        # Caso A: property específico
        span_distrito = td.find("span", {"property": "bcnbio:representingPlaceNamed"})
        if span_distrito:
            texto = span_distrito.get_text(" ", strip=True)
            m = re.search(r"(\d+)", texto)
            if m:
                distrito_num = int(m.group(1))

        # Caso B: texto genérico "Distrito"
        if distrito_num is None:
            for div in td.find_all("div"):
                texto = div.get_text(" ", strip=True)
                if "Distrito" in texto:
                    m = re.search(r"(\d+)", texto)
                    if m:
                        distrito_num = int(m.group(1))
                    break
    try:
        out["distrito"] = distrito_num
    except Exception as e:
        print(e)
    # Captura exacta de tus secciones
    fam = get_section_paragraphs(soup, [r"Familia\s+y\s+Juventud", r"Familia", r"Juventud"])
    est = get_section_paragraphs(soup, [r"Estudios\s+y\s+vida\s+laboral", r"Estudios", r"Vida\s+laboral"])
    
    out["familia_juventud_parrafos"] = fam
    out["estudios_vida_laboral_parrafos"] = est
    
    return out

def extraer_info(biografia):
    
    prompt = f"""Extrae la siguiente información biográfica desde el texto que te entregaré a continuación.
Si algún dato no está presente, responde con "desconocido" o deja el campo vacío.
Entrega la respuesta en formato JSON válido con los siguientes campos:

- lugar_nacimiento
- fecha_nacimiento
- padre
- madre
- estado_civil
- numero_total_hijos (número)
- colegios (Lista de colegios en orden)
- universidad (Solo si terminó la carrera)
- carrera (Solo si terminó la carrera)
- maximo_nivel_educativo (Enseñanza Básica, Enseñanza Media, Educación Universitaria, Magíster o Doctor/a)
- trabajo (lista trabajos)

Texto:
{biografia}
Responde solo con el JSON estructurado.
    """
    # Build the message list
    messages = [
        {"role": "user", "content": prompt}
    ]
    # Call the API (stream=False for a quick test, then you can enable streaming)
    response = client.chat(model_name, messages=messages, stream=False)
    json_str = response["message"]["content"].replace("```json", "").replace("```", "").strip()
    try:
        datos = json.loads(json_str)
    except json.JSONDecodeError as e:
        datos = {}
        
    for k, v in list(datos.items()):
        if isinstance(v, str) and v.strip().lower() == "desconocido":
            datos[k] = None
    return datos

In [49]:
for periodo in periodos:

    df_dip = pd.read_csv(DATA_DIR / periodo / 'diputados.csv')
    df_dip["nombre_norm"] = df_dip["nombre_completo"].apply(normalize_name)
    df_par["nombre_norm"] = df_par["nombre_en_lista"].apply(normalize_name)
    
    df_dip = df_dip.drop_duplicates(subset=["nombre_norm"]).reset_index(drop=True)
    
    resultados = []
    for nombre in df_dip['nombre_norm']:
        nombre_match, score = hybrid_match_nombre(nombre, df_par["nombre_norm"])
        resultados.append((nombre, nombre_match, score))

    # Crear DataFrame auxiliar con los resultados únicos
    df_match_unique = pd.DataFrame(resultados, columns=["nombre_norm", "nombre_match_norm", "score"])

    # Merge de vuelta al DataFrame original (respetando duplicados en df_dip)
    df_dip = df_dip.merge(df_match_unique, on="nombre_norm", how="left")

    # Combinar con ex parlamentarios para traer nombre y URL
    df_final = df_dip.merge(
        df_par,
        left_on="nombre_match_norm",
        right_on="nombre_norm",
        how="left"
    )
    
    df_final = (
    df_final.sort_values("score", ascending=False)
    .drop_duplicates(subset=["nombre_completo"], keep="first")
    .reset_index(drop=True)
    )

    # Reporte rápido
    n_match = (df_final["score"] >= 70).sum()
    total = len(df_final)
    print(f"Coincidencias fuertes: {n_match} de {total} ({n_match/total:.1%})")

    # Extracción de información biográfica
    res = df_final['url_wiki'].apply(extract_paragraphs_from_url)
    df_parrafos = pd.DataFrame(list(res))
    df_check = pd.concat([df_dip.reset_index(drop=True), df_parrafos], axis=1)
    
    df_check['biografia_completa'] = (
    df_check['familia_juventud_parrafos'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)) + ' ' +
    df_check['estudios_vida_laboral_parrafos'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
    ).str.strip()

    df_extraido = df_check["biografia_completa"].apply(extraer_info).apply(pd.Series)
    # Unimos al DataFrame original
    df_bio = pd.concat([df_check, df_extraido], axis=1)
    df_bio.to_csv(DATA_DIR / periodo / 'diputados_bio.csv', index=False, encoding="utf-8-sig")

Coincidencias fuertes: 157 de 157 (100.0%)


KeyError: 'biografia_completa'